# INF554 Kaggle Node2Vec Model
#### Francisco, Alex and Aksel

In [2]:
import random
import numpy as np
import pandas as pd
import csv
from tqdm import tqdm
import seaborn as sns
import os
import networkx as nx
import pdb
import pickle
from collections import Counter
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.feature_extraction import text as fe
from sklearn.decomposition import NMF, LatentDirichletAllocation
import matplotlib.pyplot as plt
import datetime
import time
import keras
import lightgbm
import spacy
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
import nltk
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin



Using TensorFlow backend.


In [4]:
with open(r"training.txt", "r") as f:
    reader = csv.reader(f)
    training  = list(reader)
# in order of training examples
training = [element[0].split(" ") for element in training]
training = pd.DataFrame(training, columns=['Node1', 'Node2', 'Link'])
print("Training examples shape: {}".format(training.shape))

with open(r"testing.txt", "r") as f:
    reader = csv.reader(f)
    testing  = list(reader)
# in order of testing examples
testing = [element[0].split(" ") for element in testing]
testing = pd.DataFrame(testing, columns=['Node1', 'Node2'])
print("Testing examples shape: {}".format(testing.shape))

Training examples shape: (453797, 3)
Testing examples shape: (113450, 2)


In [5]:
if not os.path.exists(r'pickles'):
    os.mkdir(r'pickles')

In [6]:
'''
uncomment lines for reduced corpus with stopword removal. In future integrate stemmer here, multi-language
'''
NODE_INFO_DIRECTORY = r"node_information/text/"

corpus_path = r"pickles/simple_corpus.PICKLE" 
ids_path = r"pickles/ids.PICKLE"
if os.path.exists(corpus_path):
    with open(corpus_path, 'rb') as f:
        corpus = pickle.load(f)
    f.close()
    with open(ids_path, 'rb') as f:
        ids = pickle.load(f)
    f.close()
else:
    corpus = []
    ids = []
    for filename in tqdm(os.listdir(NODE_INFO_DIRECTORY), position=0, leave=True):
        with open(NODE_INFO_DIRECTORY + filename, 'r', encoding='UTF-8', errors='ignore') as f:
            doc_string = []
            for line in f:
                [doc_string.append(token.strip()) for token in line.lower().strip().split(" ") if token != ""]
            corpus.append(' '.join(doc_string))
            ids.append(filename[:-4])
    with open(corpus_path, '+wb') as f:
        pickle.dump(corpus, f)
    f.close()
    with open(ids_path, '+wb') as f:
        pickle.dump(ids, f)
    f.close() 

In [7]:
stemmed_corpus_path = r"pickles/stemmed_corpus.PICKLE" 
if os.path.exists(stemmed_corpus_path):
    with open(stemmed_corpus_path, 'rb') as f:
        stemmed_corpus = pickle.load(f)
    f.close()
else:
    print('Stemmed corpus unavailable')

# in order of alphabetical text information i.e. 0, 1, 10, 100
node_info = pd.DataFrame({'id': ids, 'corpus': corpus, 'stemmed': stemmed_corpus})
node_info_id = node_info.set_index(['id'])
print("Training node info shape: {}".format(node_info.shape))

Training node info shape: (33226, 3)


In [8]:
train_graph_split_path = 'pickles/train_graph_split.PICKLE'

if os.path.exists(train_graph_split_path):
    with open(train_graph_split_path, 'rb') as f:
        keep_indices = pickle.load(f)
    f.close()
else:
    keep_indices = random.sample(range(len(training)), k=int(len(training) * 0.05))
    with open(train_graph_split_path, '+wb') as f:
        pickle.dump(keep_indices, f)
    f.close()

data_train_val = training.iloc[keep_indices]
data_train = training.loc[~training.index.isin(keep_indices)]

In [9]:
linked_nodes = data_train.loc[data_train['Link']=='1']
linked_nodes = linked_nodes[['Node1', 'Node2']]
linked_nodes.to_csv('linked_nodes.txt', sep=' ', index=False, header=False)
graph=nx.read_edgelist('linked_nodes.txt', create_using=nx.Graph(), nodetype = str)

In [18]:
from node2vec import Node2Vec
from gensim.models import Word2Vec

In [12]:
node2vec_path = r"pickles/NODE2VEC.PICKLE"
if os.path.exists(node2vec_path):
    print('Node2Vec already calculated')
else:
    x = ''
    while (x not in ['n', 'y']):
        x = input("Node2Vec object not available, recompute? y/n: ")
    if x == "y":
        # Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
        node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=1)

Node2Vec already calculated


In [13]:
node2vec_model_path = r"pickles/EMBEDDING_MODEL_FILENAME.model"
if os.path.exists(node2vec_model_path):
    model = Word2Vec.load(node2vec_model_path)
else:
    x = ''
    while (x not in ['n', 'y']):
        x = input("Word2Vec model not available, recompute? y/n: ")
    if x == "y":
        # Embed nodes
        model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed
        # Save embeddings for later use
        model.wv.save_word2vec_format('pickles/EMBEDDING_FILENAME.embed')
        # Save model for later use
        model.save('pickles/EMBEDDING_MODEL_FILENAME.model')

In [14]:
# Embed edges using Hadamard method
from node2vec.edges import HadamardEmbedder
edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

In [15]:
embed_size = len(edges_embs[('0', '1')])
df_train = pd.DataFrame(0, index=np.arange(len(data_train_val)), columns=range(embed_size))
j = []
for j, i in tqdm(enumerate(data_train_val.index), position=0, leave=True, total = len(data_train_val)):
    try:
        df_train.loc[j] = edges_embs[(data_train_val.loc[i]['Node1'], data_train_val.loc[i]['Node2'])]
    except:
        df_train.loc[j] = np.zeros(embed_size)

100%|███████████████████████████████████████████████████████████████████████████| 22689/22689 [00:54<00:00, 413.47it/s]


In [16]:
embed_size = len(edges_embs[('0', '1')])
df_test = pd.DataFrame(0, index=np.arange(len(testing)), columns=range(embed_size))
j = []
for i in tqdm(range(len(testing)), position=0, leave=True):
    try:
        df_test.loc[i] = edges_embs[(testing.loc[i]['Node1'], testing.loc[i]['Node2'])]
    except:
        df_test.loc[i] = np.zeros(embed_size)

100%|█████████████████████████████████████████████████████████████████████████| 113450/113450 [04:07<00:00, 458.61it/s]


In [19]:
X = df_train
y = data_train_val['Link']
y = list(map(lambda i: int(i), y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)

In [20]:
lgbm = lightgbm.LGBMClassifier()
model_lgbm = lgbm.fit(X_train, y_train)
predictions = model_lgbm.predict(X_test)
print(f1_score(y_test, predictions))

0.9082321187584345


In [106]:
lgbm = lightgbm.LGBMClassifier()
model_lgbm = lgbm.fit(X, y)
predictions = model_lgbm.predict(df_test)

In [108]:
final_pred = pd.concat([pd.DataFrame(np.arange(len(df_test))), pd.DataFrame(predictions)],axis=1)
final_pred.columns = ['id', 'predicted']
with open("node2vec_prediction.csv","+w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for index, row in final_pred.iterrows():
        csv_out.writerow(row)